# Bulk downloads and web scraping

In [ ]:
from bs4 import BeautifulSoup
import requests
import re


baseUrl = "http://www.hplovecraft.com/writings/texts/"

soup = ""

response = requests.get( baseUrl )

soup = BeautifulSoup( response.text ,"lxml")
links = soup.find_all("a")


for l in links:

    if l.get("href") is not None:
        if l.string is not None:
            linktext = l.string
        else:
            linktext = l.get("href")
            linktext = linktext[ linktext.rindex('/') + 1 : ]
            linktext = re.sub( r'[.]aspx' , '' , linktext )
        textUrl = baseUrl + l.get("href")

        if re.search( 'fiction' , textUrl ):
            print(f"{linktext}: {textUrl}")
            response = requests.get( textUrl )
            soup = BeautifulSoup( response.text ,"lxml")
            body = soup.find("body")

            fullText = body.get_text()
            fullText = re.sub( 'By H. P. Lovecraft' , ' by H. P. Lovecraft ' , fullText )

            fullText = fullText[fullText.index('About This Site') + 15 : ]
            fullText = fullText[ 0 : fullText.rindex('Return to ') ]
            fullText = fullText.strip()

            out = open( linktext + '.txt' , 'w' , encoding = 'utf-8' )
            out.write( fullText )
            out.close()


# Converting PDF to Text

In [19]:

import requests
import re

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import unicodedata, codecs
from io import StringIO

urls = '''
https://openaccess.leidenuniv.nl/bitstream/handle/1887/31806/BinnenstebuitelendeDigitalisatie.pdf
https://openaccess.leidenuniv.nl/bitstream/handle/1887/15636/Brill_2007_Het_geheim_van_de_uitgever.pdf
https://openaccess.leidenuniv.nl/bitstream/handle/1887/10375/9_62_32.pdf?sequence=1
'''

urlsList = re.split( '\s+' , str(urls).strip() ) 
for u in urlsList:
    print( '*' + u)
    if not( re.search( '\w' , u ) ):
        print( 'removed' + u)
        urlsList.remove( u )

        
        
def getPDFText(pdfFilenamePath):
    retstr = StringIO()
    parser = PDFParser(open(pdfFilenamePath,'r+b'))
    try:
        document = PDFDocument(parser)
    except Exception as e:
        print(pdfFilenamePath,'is not a readable pdf')
        return ''
    if document.is_extractable:
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr,retstr, codec='ascii' , laparams = LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
        return retstr.getvalue()
    else:
        print(pdfFilenamePath,"Warning: could not extract text from pdf file.")
        return ''




for file in urlsList:

    parts = re.split( r'/' , file )
    fileName = parts[-1]
    response = requests.get( file )
    print( response.status_code )

    if response.status_code == 200:
        open( fileName , 'wb').write( response.content)

    fullText = getPDFText( fileName )
    fileNameTxt = re.sub( r'[.]pdf$' , '.txt', fileName )
    out = open( fileNameTxt , 'w' , encoding = 'utf-8' )
    out.write( fullText )
    out.close()


*https://openaccess.leidenuniv.nl/bitstream/handle/1887/31806/BinnenstebuitelendeDigitalisatie.pdf
*https://openaccess.leidenuniv.nl/bitstream/handle/1887/15636/Brill_2007_Het_geheim_van_de_uitgever.pdf
*https://openaccess.leidenuniv.nl/bitstream/handle/1887/10375/9_62_32.pdf?sequence=1
200
200
200
